In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  
import cv2  
import os
import random
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import utils
from sklearn.metrics import accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
def load_dataset(dataset_path, target_size=(64, 64)):
    """
    * Service Name: load_dataset
    * Sync/Async: Synchronous
    * Reentrancy: Reentrant
    * Parameters (in): dataset_path , target_size
    * Parameters (inout): None
    * Parameters (out): features, labels, class_mapping
    * Return value: tuple
    * Description: Loads the dataset from the specified path, preprocesses the images
    """
    features = []
    labels = []
    class_mapping = {}

    """
    in this part we are loading the dataset from the specified path
    and preprocessing the images to extract features using HOG (Histogram of Oriented Gradients).
    then we append the features and labels to their respective lists.
    """
    class_names = sorted(os.listdir(dataset_path)) 
    for idx, class_name in enumerate(class_names):
        class_mapping[class_name] = idx  

    for class_name, label in class_mapping.items():
        class_path = os.path.join(dataset_path, class_name)
        for file_name in os.listdir(class_path):
            image_path = os.path.join(class_path, file_name)
            preprocessed_image = preprocess_image_fromPath(image_path, target_size)
            feature_vector = extract_hog_features(preprocessed_image)
            features.append(feature_vector)
            labels.append(label)

    return np.array(features), np.array(labels), class_mapping
"""
here we ar loading the dataset from the specified path
and then split the dataset into training and testing sets.
"""

dataset_path = r"D:\CUFE\GP\Sleep_detection\archive\mrleyedataset"
X, y, class_mapping = load_dataset(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Class Mapping:", class_mapping)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\CUFE\\GP\\Sleep_detection\\archive\\mrleyedataset'

: 

In [ ]:

"""
in this part we will save the data in pickle files
"""

pickle_in = open("class_mapping.pickle","rb")
class_mapping = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)



In [ ]:
import pickle

pickle_in = open("class_mapping.pickle","rb")
class_mapping = pickle.load(pickle_in)

In [ ]:

"""
in this part we will train a Support Vector Machine (SVM) model using the training data and evaluate its performance on the test data.
in which we will try to fit the SVM model to the training data and then predict the labels for the test data.
"""
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy * 100:.2f}%")


In [ ]:

filename = 'SVM_model.sav'
# pickle.dump(svm_model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)


Test the model on test images

In [ ]:
# prdicting the image
image_path = r"D:\CUFE\GP\Sleep_detection\archive\open_eye.jpg"
predicted_class = predict_image_fromPath(image_path, loaded_model, class_mapping)
print(f"Predicted Class: {predicted_class}")

Predicted Class: Open-Eyes


In [ ]:
import cv2
import dlib
import numpy as np
import time
import winsound

""" Variables to track closed eye duration """
closed_eye_start_time = None
closed_eye_duration = 0
CLOSED_EYE_THRESHOLD = 3  

""" variable to track open eye duration"""
open_eye_start_time = None
open_eye_duration = 0
OPEN_EYE_THRESHOLD = 1 

""" variables to track side face duration """
side_face_start_time = None
side_face_duration = 0
SIDE_FACE_THRESHOLD = 3 
""" Dlib face detector and predictor """
detector = dlib.get_frontal_face_detector()
#this model needed to be downloaded
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  

cap = cv2.VideoCapture(0)  
# this is important class in which we will use kalman filter to smooth the yaw angle
class KalmanFilter:
    def __init__(self):
        """
        * Service Name: KalmanFilter
        * Sync/Async: Synchronous
        * Reentrancy: Reentrant
        * Parameters (in): None
        * Parameters (inout): None
        * Parameters (out): None
        """
        self.kalman = cv2.KalmanFilter(2, 1)
        self.kalman.measurementMatrix = np.array([[1, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1, 1], [0, 1]], np.float32)
        self.kalman.processNoiseCov = np.array([[1, 0], [0, 1e-5]], np.float32)
        self.kalman.statePre = np.array([[0], [0]], np.float32)

    def update(self, measurement):
        """
        * Service Name: update
        * Sync/Async: Synchronous
        * Reentrancy: Reentrant
        * Parameters (in): measurement
        * Parameters (inout): None
        * Parameters (out): estimate
        * Return value: float
        """
        prediction = self.kalman.predict()
        estimate = self.kalman.correct(np.array([[measurement]], np.float32))
        return estimate[0][0]

kalman_filter = KalmanFilter()

while True:
    """
    how this part works:
    this part i susing Dlib to detect the face and facial landmarks
    thus it will help us to detect eyes and capture it then crops it to pass to
    the loaded SVM model to classify if the eyes are open or closed
    if the eyes arn't found it will calaculate the yaw angle of the face
    if the yaw angle is less than 30 degrees it will consider the face as frontal
    if the yaw angle is more than 30 degrees it will consider the face as side face
    it will also use a Kalman filter to smooth the yaw angle
    then if the face is frontal it will crop the eyes and eyebrows
    """
    ret, img = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        """
        in this part we will detect the face landmarks
        and calculate the yaw angle of the face
        """
        landmarks = predictor(gray, face)
        nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_ear = (landmarks.part(1).x, landmarks.part(1).y)
        right_ear = (landmarks.part(15).x, landmarks.part(15).y)

        # Calculate the distance from the nose to both ears
        left_distance = np.linalg.norm(np.array(nose) - np.array(left_ear))
        right_distance = np.linalg.norm(np.array(nose) - np.array(right_ear))
        
        # Calculate yaw using ratio of distances
        yaw_ratio = right_distance / (left_distance + right_distance)
        yaw_angle = (yaw_ratio - 0.5) * 90  # Convert ratio to degrees (-45 to 45)
        yaw_angle = kalman_filter.update(yaw_angle)  # Apply Kalman filter

        # Determine face orientation
        if abs(yaw_angle) < 30:
            face_orientation = "Frontal Face"
            left_eye_pts = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(17, 22)] +  # Left eyebrow
                                    [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]+  # Left eye
                                    [(landmarks.part(n).x, landmarks.part(n).y) for n in range(28, 31)])  
                                    
            right_eye_pts = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(22, 27)] +  # Right eyebrow
                                    [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]+  # Right eye
                                    [(landmarks.part(n).x, landmarks.part(n).y) for n in range(29, 31)]
                                    )  
            x_left, y_left, w_left, h_left = cv2.boundingRect(left_eye_pts)
            x_right, y_right, w_right, h_right = cv2.boundingRect(right_eye_pts)
            expand_ratio = 0.2
            h_expand = int(h_left * expand_ratio)  
            y_left = max(y_left - h_expand, 0)
            h_left = min(h_left + h_expand, img.shape[0] - y_left)
            y_right = max(y_right - h_expand, 0)
            h_right = min(h_right + h_expand, img.shape[0] - y_right)

            """
            here we are cropping the eyes from the image
            and saving them to the archive folder
            """
            left_eye = img[y_left:y_left+h_left, x_left:x_left+w_left]
            right_eye = img[y_right:y_right+h_right, x_right:x_right+w_right]
            cv2.imwrite("archive/person_eye_left.jpg", left_eye)
            cv2.imwrite("archive/person_eye_right.jpg", right_eye)
            cv2.rectangle(img, (x_left, y_left), (x_left + w_left, y_left + h_left), (0, 255, 0), 2)
            cv2.rectangle(img, (x_right, y_right), (x_right + w_right, y_right + h_right), (0, 255, 0), 2)

            """
            then we will pass the cropped eyes to the loaded SVM model
            to predict if the eyes are open or closed
            """
            try:
                left_eye_class = predict_image_fromPath("archive/person_eye_left.jpg", loaded_model, class_mapping)
                right_eye_class = predict_image_fromPath("archive/person_eye_right.jpg", loaded_model, class_mapping)
            except:
                print("Error in prediction")
                continue

            """
            in this part of the code we will check if the eyes are closed or open
            if both eyes are closed we will start timing the closed eyes
            else if one of the eyes is opened then the eyes are considered open
            if the eyes are closed for more than 3 seconds we will beep a sound
            if the eyes are open for more than 1 second we will reset the closed eyes timer
            """
            
            """
            we are checking the classes of the left and right eyes if there are both closed
            """
            if left_eye_class == "Close-Eyes" and right_eye_class == "Close-Eyes":
                print("Close-Eyes")
                if closed_eye_start_time is None:
                    closed_eye_start_time = time.time()
                
                # Calculate duration of closed eyes
                closed_eye_duration = time.time() - closed_eye_start_time
                # here we checks if closed eye duration exceeds threshold
                if closed_eye_duration >= CLOSED_EYE_THRESHOLD:
                    print(f"Eyes closed for {closed_eye_duration:.2f} seconds")
                    winsound.Beep(2500, 1000)  
                    # Resetting timing parameters
                    open_eye_start_time = None
                    open_eye_duration = 0
                    side_face_start_time = None
                    side_face_duration = 0

            else:
                """
                here we are checking if the eyes are open
                then reseting the closed eye timer
                """
                
                print(f"Eyes open for {open_eye_duration:.2f} seconds")
                # Start or continue timing open eyes
                if open_eye_start_time is None:
                    open_eye_start_time = time.time()
                # Calculate duration of open eyes
                open_eye_duration = time.time() - open_eye_start_time
                # Check if open eye duration exceeds threshold
                if open_eye_duration >= OPEN_EYE_THRESHOLD:  
                    # Reseting Timing Parameters
                    closed_eye_start_time = None
                    closed_eye_duration = 0
                    side_face_start_time = None
                    side_face_duration = 0

        else:
            """
            here we are checking if the face is turned to the side
            if the yaw angle is more than 30 degrees
            we will start timing the side face
            """
            face_orientation = "Side Face"
            if side_face_start_time is None:
                side_face_start_time = time.time()
            side_face_duration = time.time() - side_face_start_time
            if side_face_duration >= SIDE_FACE_THRESHOLD:
                print(f"Face turned to the side for {side_face_duration:.2f} seconds")
                winsound.Beep(2500, 1000)
                # Resetting timers
                open_eye_start_time = None
                open_eye_duration = 0
                closed_eye_start_time = None
                closed_eye_duration = 0

        
        print(f"Face Orientation: {face_orientation} (Yaw: {yaw_angle:.2f}°)")
        cv2.putText(img, face_orientation, (face.left(), face.top() - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)


    # Show real-time webcam feed
    cv2.imshow("Driver Monitoring", img)
    
    # Press `Esc` to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction
Error in prediction


KeyboardInterrupt: 

In [ ]:
test = predict_image_fromPath("archive/person_eye_left.jpg", loaded_model, class_mapping)
print(f"Predicted Class: {test}")
test = predict_image_fromPath("archive/person_eye_right.jpg", loaded_model, class_mapping)
print(f"Predicted Class: {test}")

Predicted Class: Close-Eyes
Predicted Class: Open-Eyes


In [ ]:
# === Importing Required Libraries ===
# These libraries provide tools for image processing, machine learning, and face landmark detection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import numpy as np
import argparse
import imutils
import cv2  # OpenCV for real-time image processing
import os
import random
import matplotlib.pyplot as plt
from skimage.feature import hog  # For HOG feature extraction
from skimage import data, exposure
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle  # To load pre-trained model and class labels
import dlib  # For facial landmark detection
import time  # To measure time intervals for sleep detection
import winsound  # For beeping alerts (works on Windows)

# === Image Preprocessing Function ===
def preprocess_image_fromPath(image_path, target_size=(64, 64)):
    """
    Loads an image from disk, converts it to grayscale, smooths it,
    resizes it, and normalizes the pixel values for feature extraction.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")
    
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian and Median Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    filtered = cv2.medianBlur(blurred, 3)

    # Resize to target size (64x64)
    resized = cv2.resize(filtered, target_size)
    
    # Normalize pixel values to [0, 1]
    normalized = resized / 255.0
    return normalized

# === HOG Feature Extraction Function ===
def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    Extract Histogram of Oriented Gradients (HOG) features from an image.
    """
    hog_features = hog(image, orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm='L2-Hys', visualize=False)
    return hog_features

# === Load Pretrained SVM Model and Class Mapping ===
# Load the class mapping: {'Open-Eyes': 1, 'Close-Eyes': 0}
pickle_in = open("class_mapping.pickle", "rb")
class_mapping = pickle.load(pickle_in)

# Load trained SVM model for eye classification
filename = 'SVM_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# === Image Classification Function ===
def predict_image_fromPath(image_path, model, class_mapping, target_size=(64, 64)):
    """
    Predict the class of a given image using the SVM model and return class name.
    """
    preprocessed_image = preprocess_image_fromPath(image_path, target_size)
    feature_vector = extract_hog_features(preprocessed_image).reshape(1, -1)
    prediction = model.predict(feature_vector)[0]
    
    # Convert class index to class name using class_mapping
    for class_name, label in class_mapping.items():
        if label == prediction:
            return class_name

# === Kalman Filter Class to Smooth Head Pose Angles ===
class KalmanFilter:
    """
    Applies a Kalman filter to smooth noisy yaw angle values over time.
    """
    def __init__(self):
        self.kalman = cv2.KalmanFilter(2, 1)  # 2 state variables, 1 measurement
        self.kalman.measurementMatrix = np.array([[1, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1, 1], [0, 1]], np.float32)
        self.kalman.processNoiseCov = np.array([[1, 0], [0, 1e-5]], np.float32)
        self.kalman.statePre = np.array([[0], [0]], np.float32)

    def update(self, measurement):
        prediction = self.kalman.predict()
        estimate = self.kalman.correct(np.array([[measurement]], np.float32))
        return estimate[0][0]

# === Quick Test ===
# Test the classifier on a sample image
image_path = r"D:\CUFE\GP\Sleep_detection\archive\open_eye.jpg"
predicted_class = predict_image_fromPath(image_path, loaded_model, class_mapping)
print(f"Predicted Class: {predicted_class}")

# === Time Thresholds and State Variables for Monitoring ===
closed_eye_start_time = None
closed_eye_duration = 0
CLOSED_EYE_THRESHOLD = 3  # Raise alert if eyes closed for >= 3 seconds

open_eye_start_time = None
open_eye_duration = 0
OPEN_EYE_THRESHOLD = 1  # Reset states if eyes open for >= 1 second

side_face_start_time = None
side_face_duration = 0
SIDE_FACE_THRESHOLD = 3  # Raise alert if face turned for >= 3 seconds

# === Facial Landmark Detection Setup ===
# Load dlib’s face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# === Start Webcam Capture ===
cap = cv2.VideoCapture(0)

# Create instance of the Kalman filter
kalman_filter = KalmanFilter()

# === Main Loop for Real-time Monitoring ===
while True:
    ret, img = cap.read()
    if not ret:
        break

    # Convert to grayscale for facial detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    for face in faces:
        # Get facial landmarks (68 points)
        landmarks = predictor(gray, face)

        # Extract nose and ears to estimate face yaw (left/right orientation)
        nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_ear = (landmarks.part(1).x, landmarks.part(1).y)
        right_ear = (landmarks.part(15).x, landmarks.part(15).y)

        # Estimate yaw angle based on distances from nose to ears
        left_distance = np.linalg.norm(np.array(nose) - np.array(left_ear))
        right_distance = np.linalg.norm(np.array(nose) - np.array(right_ear))
        yaw_ratio = right_distance / (left_distance + right_distance)
        yaw_angle = (yaw_ratio - 0.5) * 90  # Approximate -45 to 45°
        yaw_angle = kalman_filter.update(yaw_angle)  # Smooth it

        # --- Check if face is forward or turned sideways ---
        if abs(yaw_angle) < 30:
            face_orientation = "Frontal Face"

            # Extract eye region coordinates from landmarks
            left_eye_pts = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
            right_eye_pts = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

            # Get bounding box for both eyes
            x_left, y_left, w_left, h_left = cv2.boundingRect(left_eye_pts)
            x_right, y_right, w_right, h_right = cv2.boundingRect(right_eye_pts)

            # Slightly expand bounding box to include full eye
            expand_ratio = 0.2
            h_expand = int(h_left * expand_ratio)
            y_left = max(y_left - h_expand, 0)
            h_left = min(h_left + h_expand, img.shape[0] - y_left)
            y_right = max(y_right - h_expand, 0)
            h_right = min(h_right + h_expand, img.shape[0] - y_right)

            # Crop eye images from original frame
            left_eye = img[y_left:y_left+h_left, x_left:x_left+w_left]
            right_eye = img[y_right:y_right+h_right, x_right:x_right+w_right]

            # Save temporary images of the eyes
            cv2.imwrite("archive/person_eye_left.jpg", left_eye)
            cv2.imwrite("archive/person_eye_right.jpg", right_eye)

            # Draw rectangles for visualization
            cv2.rectangle(img, (x_left, y_left), (x_left + w_left, y_left + h_left), (0, 255, 0), 2)
            cv2.rectangle(img, (x_right, y_right), (x_right + w_right, y_right + h_right), (0, 255, 0), 2)

            # Predict if eyes are open or closed
            try:
                left_eye_class = predict_image_fromPath("archive/person_eye_left.jpg", loaded_model, class_mapping)
                right_eye_class = predict_image_fromPath("archive/person_eye_right.jpg", loaded_model, class_mapping)
            except:
                print("Error in prediction")
                continue

            # === Both Eyes Closed ===
            if left_eye_class == "Close-Eyes" and right_eye_class == "Close-Eyes":
                print("Close-Eyes")
                if closed_eye_start_time is None:
                    closed_eye_start_time = time.time()
                closed_eye_duration = time.time() - closed_eye_start_time

                if closed_eye_duration >= CLOSED_EYE_THRESHOLD:
                    print(f"Eyes closed for {closed_eye_duration:.2f} seconds")
                    winsound.Beep(2500, 1000)  # Beep alert
                    # Reset other timers
                    open_eye_start_time = None
                    open_eye_duration = 0
                    side_face_start_time = None
                    side_face_duration = 0

            # === Eyes Open ===
            else:
                print(f"Eyes open for {open_eye_duration:.2f} seconds")
                if open_eye_start_time is None:
                    open_eye_start_time = time.time()
                open_eye_duration = time.time() - open_eye_start_time

                if open_eye_duration >= OPEN_EYE_THRESHOLD:
                    # Reset closed eye and side face timers
                    closed_eye_start_time = None
                    closed_eye_duration = 0
                    side_face_start_time = None
                    side_face_duration = 0

        # === Face Turned Away (Yaw > 30°) ===
        else:
            face_orientation = "Side Face"
            if side_face_start_time is None:
                side_face_start_time = time.time()
            side_face_duration = time.time() - side_face_start_time

            if side_face_duration >= SIDE_FACE_THRESHOLD:
                print(f"Face turned for {side_face_duration:.2f} seconds")
                winsound.Beep(2500, 1000)  # Alert
                # Reset other timers
                open_eye_start_time = None
                open_eye_duration = 0
                closed_eye_start_time = None
                closed_eye_duration = 0

        # Show orientation text on screen
        print(f"Face Orientation: {face_orientation} (Yaw: {yaw_angle:.2f}°)")
        cv2.putText(img, face_orientation, (face.left(), face.top() - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # === Display Webcam Frame ===
    cv2.imshow("Driver Monitoring", img)
    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

# === Cleanup ===
cap.release()
cv2.destroyAllWindows()
